In [9]:
import os
import random
import sys
import platform
import cv2

import pandas as pd
from sklearn.impute import SimpleImputer

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import networkx as nx
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from scipy.stats import zscore
import seaborn as sns
from scipy.spatial.distance import euclidean
import math
from matplotlib.patches import Wedge, Ellipse, Polygon, Circle
import pickle
import shutil 

def check_os():
    os = platform.system()

    if os == "Darwin":
        return "MacOS"
    elif os == "Linux":
        return "Linux"
    else:
        return "Unknown OS"

operating_system = check_os()

if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [10]:
df = pd.read_csv("/Users/johnny/Projects/Master_Thesis_CV/datasets/annotations.csv")

In [11]:
with open("/Users/johnny/Projects/Master_Thesis_CV/datasets/coord_dict_smooth.pkl", "rb") as f:
    coord_dict = pickle.load(f)

In [13]:
with open('id112.npy', 'wb') as f:
    np.save(np.array(coord_dict[112]))

array([[130,  91],
       [133,  91],
       [134,  93],
       [136,  93],
       [137,  93],
       [139,  94],
       [141,  94],
       [142,  95],
       [144,  95],
       [147,  95],
       [149,  97],
       [151,  96],
       [152,  97],
       [153,  97],
       [154,  98],
       [156,  99],
       [157,  99],
       [159,  99],
       [159,  99],
       [161, 100],
       [164, 101],
       [166, 102],
       [168, 102],
       [168, 103],
       [168, 103],
       [171, 104],
       [172, 106],
       [175, 107],
       [190, 111],
       [192, 109],
       [195, 111],
       [198, 111],
       [200, 112],
       [203, 112],
       [206, 114],
       [210, 115],
       [214, 115],
       [217, 116],
       [216, 116],
       [215, 117],
       [217, 119],
       [218, 121],
       [223, 121],
       [226, 123],
       [230, 124],
       [237, 124],
       [248, 124],
       [264, 123],
       [273, 122],
       [284, 122],
       [291, 121],
       [298, 122],
       [304,

In [20]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2

# if exists trajectories folder delete and create it
if os.path.exists("./trajectories/"):
    shutil.rmtree("./trajectories/")
os.makedirs("./trajectories/")

filtered_df = df[df.folder == 20210415]

# Construye un diccionario {clip_name: {id: trajectory}}
clip_dict = {}
for index, row in filtered_df.iterrows():
    clip_name = row['clip']
    id = row['id']
    coord = [row['x'], row['y']]
    
    if clip_name not in clip_dict:
        clip_dict[clip_name] = {}
    
    if id not in clip_dict[clip_name]:
        clip_dict[clip_name][id] = []

    clip_dict[clip_name][id].append(coord)

# Filtra las trayectorias que están en coord_dict
for clip_name, trajectories in clip_dict.items():
    clip_dict[clip_name] = {k: v for k, v in trajectories.items() if k in coord_dict}

# Plot y guarda las trayectorias agrupadas por clip
for clip_name, trajectories in clip_dict.items():
    clip_path = f"./trajectories/{clip_name}"
    os.makedirs(clip_path, exist_ok=True)  # Crea una subcarpeta para cada clip
    
    # Aquí supongo que tienes una estructura de nombres de archivo similar para tus imágenes
    # Cambia esta línea según cómo estén organizados tus datos
    img_path = root_path + f"datasets/harbor-synthetic/LTD_Dataset/LTD_Dataset/Image_Dataset_25fps/20200514/clip_15_2004/image_0100.jpg"
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    for k, v in trajectories.items():
        v_array = np.array(v)  # Convert v to a NumPy array
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for i in range(1, len(v_array)):
            start_point = tuple(v_array[i - 1].astype(int))
            end_point = tuple(v_array[i].astype(int))
            img_rgb = cv2.line(img_rgb, start_point, end_point, (255, 0, 0), 1)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.title(f"ID: {k}")
        plt.savefig(f"{clip_path}/{k}.png")
        plt.close()
